## XGBoost for Classification Problems
______

- XGBoost is a library created for classification problems
- Its speed and performance are unparalleled
- It consistently outperforms any other algorithms aimed at supervised learning tasks
- Requirements for XGBoost to achieve top performance:
    - Numeric features should be scaled
    - Categorical features should be encoded

To show how these steps are done, we will be using the Rain in Australia dataset from Kaggle where we will predict whether it will rain today or not based on some weather measurements. In this section, we will focus on preprocessing by utilizing Scikit-Learn Pipelines.

In [1]:
import pandas as pd

rain = pd.read_csv("data/weatherAUS.csv")

rain.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [2]:
rain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

The dataset contains weather measures of 10 years from multiple weather stations in Australia. You can either predict whether it will rain tomorrow or today, so there are two targets in the dataset named RainToday, RainTomorrow.

Since we will only be predicting for RainToday, we will drop the other one along with some other features that won't be necessary:

In [3]:
rain.drop(["Date", "Location", "RainTomorrow", "Rainfall"], axis=1, inplace=True)

Next, let’s deal with missing values starting by looking at their proportions in each column:

In [4]:
missing_props = rain.isna().mean(axis=0)
missing_props

MinTemp          0.010209
MaxTemp          0.008669
Evaporation      0.431665
Sunshine         0.480098
WindGustDir      0.070989
WindGustSpeed    0.070555
WindDir9am       0.072639
WindDir3pm       0.029066
WindSpeed9am     0.012148
WindSpeed3pm     0.021050
Humidity9am      0.018246
Humidity3pm      0.030984
Pressure9am      0.103568
Pressure3pm      0.103314
Cloud9am         0.384216
Cloud3pm         0.408071
Temp9am          0.012148
Temp3pm          0.024811
RainToday        0.022419
dtype: float64

If the proportion is higher than 40% we will drop the column:

In [5]:
over_threshold = missing_props[missing_props >= 0.4]
over_threshold

Evaporation    0.431665
Sunshine       0.480098
Cloud3pm       0.408071
dtype: float64

Three columns contain more than 40% missing values. We will drop them:

In [6]:
rain.drop(over_threshold.index, axis=1, inplace=True)

Now, before we move on to pipelines, let’s divide the data into feature and target arrays beforehand:

In [7]:
X = rain.drop("RainToday", axis=1)
y = rain.RainToday

In [8]:
y = y.map(lambda x: 0 if x=='No' else 1)

Next, there are both categorical and numeric features. We will build two separate pipelines and combine them later.

For the categorical features, we will impute the missing values with the mode of the column and encode them with `One-Hot encoding`:

In [9]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

categorical_pipeline = Pipeline(
    steps=[
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("oh-encode", OneHotEncoder(handle_unknown="ignore", sparse=False)),
    ]
)

For the numeric features, I will choose the mean as an imputer and `StandardScaler` so that the features have 0 mean and a variance of 1:

In [10]:
from sklearn.preprocessing import StandardScaler

numeric_pipeline = Pipeline(
    steps=[("impute", SimpleImputer(strategy="mean")), 
           ("scale", StandardScaler())]
)

Finally, we will combine the two pipelines with a column transformer. To specify which columns the pipelines are designed for, we should first isolate the categorical and numeric feature names:

In [11]:
cat_cols = X.select_dtypes(exclude="number").columns
num_cols = X.select_dtypes(include="number").columns

Next, we will input these along with their corresponding pipelines into a `ColumnTransFormer` instance:

In [12]:
from sklearn.compose import ColumnTransformer

full_processor = ColumnTransformer(
    transformers=[
        ("numeric", numeric_pipeline, num_cols),
        ("categorical", categorical_pipeline, cat_cols),
    ]
)

The full pipeline is finally ready. Now we'll add an `XGBoost` classifier.

We can import it under its standard alias — xgb. For classification problems, the library provides XGBClassifier class.

In [13]:
import xgboost as xgb

xgb_cl = xgb.XGBClassifier()

print(type(xgb_cl))

<class 'xgboost.sklearn.XGBClassifier'>


Fortunately, the classifier follows the familiar fit-predict pattern of sklearn meaning we can freely use it as any sklearn model.

Before we train the classifier, let’s preprocess the data and divide it into train and test sets:

In [14]:
# Apply preprocessing
X_processed = full_processor.fit_transform(X)
y_processed = SimpleImputer(strategy="most_frequent").fit_transform(y.values.reshape(-1, 1))

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_processed, y_processed, stratify=y_processed, random_state=1121218)

Since the target contains NaN, I imputed it by hand. Also, it is important to pass y_processed to stratify so that the split contains the same proportion of categories in both sets.

Now, we fit the classifier with default parameters and evaluate its performance:

In [15]:
from sklearn.metrics import accuracy_score

# Init classifier
xgb_cl = xgb.XGBClassifier()

# Fit
xgb_cl.fit(X_train, y_train)

# Predict
preds = xgb_cl.predict(X_test)

# Score
accuracy_score(y_test, preds)

0.8368761171456071